In [1]:
import pandas as pd
import numpy as np
import json
import datetime
import os
from haversine import haversine
import gc

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

print(datetime.datetime.now())
os.getcwd()


2019-10-29 14:50:21.413670


'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/JT_BR_zips_by_Store_type_and_Dentisty_20191028'

In [2]:
ExcelFile=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/Copy of BL 2019Q4 Budget Allocation Model 20191018_JC_JL_20191020.xlsx")
ExcelFile.sheet_names

['parameters',
 'Closed 12 mo thru July (57)',
 "Closing Aug-Oct '19 (12)",
 'SOTF pre 11-15-19 (462)',
 'SOTF New,Relo Oct-Nov SOTF (11)',
 'all_PS10_zips_BL',
 '1_raw',
 '1_1',
 '2_raw_2018Q4filled',
 '3_sales%_budget',
 '4_SOTF',
 'Sheet2',
 '5_Rewards',
 '6_closed_Quad3',
 '7_split',
 'HML_PS_Zip 12 Mo',
 'HML_PS_Zip 18 mo',
 'Summary Chart 12 mo',
 'Summary Chart 18mo',
 'Zips by Density and PS 18mo',
 'Stores by Density 18mo',
 'Sheet1',
 '7_split 9mo V2']

In [3]:
# location names changed for the convience
df_store_density=ExcelFile.parse("Summary Chart 18mo",usecols=["location_id","Tier"],dtype=str)
df_store_density=df_store_density.rename(columns={"Tier":"Density_Tier"})


df_SOTF=ExcelFile.parse("4_SOTF",usecols=["location_id","copy_SOTF_label"],dtype=str)
df_SOTF=df_SOTF.rename(columns={"copy_SOTF_label":"SOTF"})
df_SOTF['SOTF']=df_SOTF['SOTF'].replace("nan",np.nan)

del ExcelFile
gc.collect()



88216

In [4]:
list_missing_stores=df_SOTF[~df_SOTF['location_id'].isin(df_store_density['location_id'].tolist())]['location_id'].tolist()
print(len(list_missing_stores))
# Checked a closed or consistant quadrant III stores

26


In [5]:
df_zip_by_store=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
df_zip_by_store.sheet_names

['unique_zips_full_footprint',
 'view_by_TA',
 'view_by_store',
 'view_for_Tableau']

In [6]:
df_zip_by_store=df_zip_by_store.parse("view_by_store",dtype=str)
print(df_zip_by_store.shape)
df_zip_by_store.head(3)

(1408, 17)


,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."
2,1,multiple_1,4543,3601 LAKEVIEW PKWY,NaN,75088,ROWLETT,TX,32.907788,-96.566747,{'DALLAS-FT. WORTH'},"{'DALLAS', 'ROCKWALL'}","['75089', '75088', '75040']","['75043', '75087', '75041', '75032', '75189', ...",[],"['75040', '75088', '75089']","['75087', '75032', '75042', '75150', '75048', ..."


In [7]:
df_zip_by_store=df_zip_by_store[['location_id','all_trans_P_zips','all_trans_S_zips','zips_in_10']].reset_index()
del df_zip_by_store['index']
df_output_by_zip=pd.DataFrame()
for ind,row in df_zip_by_store.iterrows():
    store_num=row['location_id']
    zip_P_list=eval(row['all_trans_P_zips'])
    zip_S_list=eval(row['all_trans_S_zips'])
    zip_10_list=eval(row['zips_in_10'])
    
    df_P=pd.DataFrame({"zip_cd":zip_P_list},index=['P']*len(zip_P_list)).reset_index().rename(columns={"index":"zip_label"})
    df_S=pd.DataFrame({"zip_cd":zip_S_list},index=['S']*len(zip_S_list)).reset_index().rename(columns={"index":"zip_label"})
    df_10=pd.DataFrame({"zip_cd":zip_10_list},index=['zip_in_10']*len(zip_10_list)).reset_index().rename(columns={"index":"zip_label"})
    
    df=df_P.append(df_S).append(df_10)
    df['location_id']=store_num
    df_output_by_zip=df_output_by_zip.append(df)
df_output_by_zip.shape

(25132, 3)

In [8]:
df_store_density.head(2)

,location_id,Density_Tier
0,5326,H
1,1786,H


In [9]:
df_output=pd.merge(df_store_density,df_output_by_zip,on="location_id",how="left")
df_output_existing=df_output[~pd.isnull(df_output['zip_label'])]
df_output_new=df_output[pd.isnull(df_output['zip_label'])]

In [10]:
store_location=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20191001-134050-499.txt",sep="|",dtype=str)
store_location=store_location[['location_id','latitude_meas','longitude_meas']]
store_location['latitude_meas']=store_location['latitude_meas'].astype(float)
store_location['longitude_meas']=store_location['longitude_meas'].astype(float)
df_output_new=pd.merge(df_output_new,store_location,on="location_id",how="left")
df_output_new_0=df_output_new[pd.notnull(df_output_new['latitude_meas'])]
df_output_new_1=df_output_new[pd.isnull(df_output_new['latitude_meas'])]

In [11]:
'''
recent_store=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-10-19.csv",dtype=str)
recent_store=recent_store[recent_store['week_end_date'].apply(lambda x: x[:7]=="2019-10")]
recent_store['sales']=recent_store['sales'].astype(float)
recent_store=recent_store[recent_store['sales']>0]
print(recent_store.shape,recent_store['week_end_date'].nunique())

recent_store[recent_store['location_id'].isin(list_missing_stores)]

#mentioned above, all closed or Quad III in BR's excel
'''

'\nrecent_store=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-10-19.csv",dtype=str)\nrecent_store=recent_store[recent_store[\'week_end_date\'].apply(lambda x: x[:7]=="2019-10")]\nrecent_store[\'sales\']=recent_store[\'sales\'].astype(float)\nrecent_store=recent_store[recent_store[\'sales\']>0]\nprint(recent_store.shape,recent_store[\'week_end_date\'].nunique())\n\nrecent_store[recent_store[\'location_id\'].isin(list_missing_stores)]\n\n#mentioned above, all closed or Quad III in BR\'s excel\n'

In [12]:
dict_new_store_address={}

# 5399 	851 SOUTH 30TH STREET	INDIAN MOUND MALL	HEATH	OH	43056
dict_new_store_address.update({"5399":{"GoogleAddress":"771 S 30th St Ste 9001, Heath, OH 43056","GoogleCoor":[40.0312042,-82.4473616]}})

# 5400 	2235 EAST STATE STREET	HERMITAGE TOWN PLAZA	HERMITAGE	PA	16148-2727
dict_new_store_address.update({"5400":{"GoogleAddress":"2253 E State St, Hermitage, PA 16148","GoogleCoor":[41.234349,-80.4658798]}})

# 4711 	4655 CANAL AVENUE SOUTHWEST	GRANDVILLE MARKETPLACE	GRANDVILLE	MI	49418-8731
dict_new_store_address.update({"4711":{"GoogleAddress":"4485-4655 Canal Ave SW, Grandville, MI 49418","GoogleCoor":[42.8815927,-85.7745291]}})

# 5411 	2318 WEST MERCURY BOULEVARD	TODD'S LANE SHOPS	HAMPTON	VA	23666
dict_new_store_address.update({"5411":{"GoogleAddress":"2318 W Mercury Blvd, Hampton, VA 23666","GoogleCoor":[37.0394621,-76.4073108]}})

# 4709 	5252 BARDSTOWN ROAD	BARDSTOWN PLACE	LOUISVILLE	KY	40291-1766
dict_new_store_address.update({"4709":{"GoogleAddress":"5252 Bardstown Rd, Louisville, KY 40291","GoogleCoor":[38.1680876,-85.6073362]}})

# 5410 	9690 COLERAIN AVENUE	PROSPECT SQUARE	CINCINNATI	OH	45251
dict_new_store_address.update({"5410":{"GoogleAddress":"9690 Colerain Ave, Cincinnati, OH 45251","GoogleCoor":[39.2482448,-84.5948913]}})

# 4707 	14215 SOUTHEAST PETROVITSKY ROAD	FAIRWOOD SQUARE	RENTON	WA	98058-8983
dict_new_store_address.update({"4707":{"GoogleAddress":"14215 SE Petrovitsky Rd, Renton, WA 98058","GoogleCoor":[47.4444178,-122.1519844]}})

# 4701 	5055 W SAHARA AVE	SAHARA DECATUR	LAS VEGAS	NV	89146-3405
dict_new_store_address.update({"4701":{"GoogleAddress":"5055 W Sahara Ave, Las Vegas, NV 89146","GoogleCoor":[36.1436122,-115.2103361]}})

# 5409 	4585 EASTGATE BOULEVARD	TOYS R US CENTER	CINCINNATI	OH	45245-1201
dict_new_store_address.update({"5409":{"GoogleAddress":"4585 Eastgate Blvd, Cincinnati, OH 45245","GoogleCoor":[39.102485,-84.2740534]}})

# 4705 	370 AUTO CENTER CIRCLE	NORTHRIDGE MALL	SALINAS	CA	93907
dict_new_store_address.update({"4705":{"GoogleAddress":"370 Auto Center Cir, Salinas, CA 93906","GoogleCoor":[36.7148622,-121.6607234]}})

# 4706 	3170 CHILI AVE UNIT # T5	CHILI PLAZA	ROCHESTER	NY	14624-5410
dict_new_store_address.update({"4706":{"GoogleAddress":"3170 Chili Ave T5, Rochester, NY 14624","GoogleCoor":[43.1085609,-77.7467213]}})

# 4702 	6375 MONTANA AVENUE	KMART SHOPPING CENTER	EL PASO	TX	79925-2044
dict_new_store_address.update({"4702":{"GoogleAddress":"6375 Montana Ave, El Paso, TX 79925","GoogleCoor":[31.7898616,-106.404997]}})

# 4678 	3501 EAST LINCOLNWAY	CHEYENNE PLAZA	CHEYENNE	WY	82001-6302
dict_new_store_address.update({"4678":{"GoogleAddress":"3501 E Lincolnway, Cheyenne, WY 82001","GoogleCoor":[41.1370675,-104.7674035]}})

# 5407 	14154 EAST WADE HAMPTON BLVD	COLLINS CENTER	GREER	SC	29651-1554
dict_new_store_address.update({"5407":{"GoogleAddress":"14154 E Wade Hampton Blvd, Greer, SC 29651","GoogleCoor":[34.948923,-82.2118793]}})

# 5402 	14603 TELEGRAPH ROAD	TOYS R US CENTER	WOODBRIDGE	VA	22192-4601
dict_new_store_address.update({"5402":{"GoogleAddress":"14603 Telegraph Rd, Woodbridge, VA 22194","GoogleCoor":[38.6373899,-77.2914551]}})

# 5398 	275 ROUTE 18	MIRACLE MALL	EAST BRUNSWICK	NJ	08816-1904
dict_new_store_address.update({"5398":{"GoogleAddress":"275 NJ-18, East Brunswick, NJ 08816","GoogleCoor":[40.4592229,-74.4049486]}})

# 5408 	3420 SOUTHWEST DURHAM DRIVE	PATTERSON PLACE	DURHAM	NC	27707-3351
dict_new_store_address.update({"5408":{"GoogleAddress":"3420 SW Durham Dr, Durham, NC 27707","GoogleCoor":[35.9516434,-78.9887268]}})

In [13]:
df_output_new_1['latitude_meas']=df_output_new_1['location_id'].apply(lambda x: dict_new_store_address[x]['GoogleCoor'][0])
df_output_new_1['longitude_meas']=df_output_new_1['location_id'].apply(lambda x: dict_new_store_address[x]['GoogleCoor'][1])


df_output_new=df_output_new_0.append(df_output_new_1)
del df_output_new_0
del df_output_new_1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df_output_new=df_output_new.reset_index()
del df_output_new['index']
df_zip_new_store=pd.DataFrame()
for ind,row in df_output_new.iterrows():
    store_center=(row['latitude_meas'],row['longitude_meas'])
    df_store=pd.DataFrame()
    for zip_cd,v_centroid in zip_centers.items():
        dist=haversine(store_center,v_centroid,unit="mi")
        if dist<=10:
            df=pd.DataFrame({"zip_cd":zip_cd},index=['zip_in_10'])
            df_store=df_store.append(df)
    df_store=df_store.reset_index().rename(columns={"index":"zip_label"})
    df_store['location_id']=row['location_id']
    df_zip_new_store=df_zip_new_store.append(df_store)
df_zip_new_store.head(2)
            
        

,zip_label,zip_cd,location_id
0,zip_in_10,26142,4698
1,zip_in_10,45713,4698


In [15]:
del df_output_new['zip_label']
del df_output_new['zip_cd']
df_output_new=pd.merge(df_output_new,df_zip_new_store,on="location_id",how="left")
df_output_new.shape

(764, 6)

In [16]:
df_output_new=df_output_new[['location_id','Density_Tier','zip_label','zip_cd']].drop_duplicates()
df_output_new.shape

(764, 4)

In [17]:
df_output=df_output_existing.append(df_output_new)

In [18]:
df_output=pd.merge(df_output,df_SOTF,on="location_id",how="left")
df_output['SOTF']=df_output['SOTF'].fillna("Legacy")
df_output.shape

(25450, 5)

In [19]:
df_output.head(2)

,location_id,Density_Tier,zip_label,zip_cd,SOTF
0,5326,H,P,08332,Legacy
1,5326,H,P,08360,Legacy


In [20]:
writer=pd.ExcelWriter("./BL_zips_by_store_density_and_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"zip_by_store",index=False)


In [22]:
df_output.shape

(25450, 5)

In [23]:
for density_sotf,group in df_output.groupby(['Density_Tier','SOTF']):
    if density_sotf[0]!="M" or density_sotf[1]!="SOTF":
        tab_name=density_sotf[0]+"_"+density_sotf[1]

        df=group[['Density_Tier','SOTF','zip_cd']].drop_duplicates()
        df.to_excel(writer,tab_name,index=False)
        print(tab_name,df.shape)
    else:
        for zip_label,df_inner_group in group.groupby("zip_label"):
            tab_name=density_sotf[0]+"_"+density_sotf[1]+"_"+zip_label
            df=df_inner_group[['Density_Tier','SOTF','zip_cd','zip_label']].drop_duplicates()
            df.to_excel(writer,tab_name,index=False)
            print(tab_name,df.shape)
            

H_Legacy (1804, 3)
L_Legacy (55, 3)
L_SOTF (1937, 3)
M_Legacy (8823, 3)
M_SOTF_P (1477, 4)
M_SOTF_S (3761, 4)


In [24]:
writer.save()

In [23]:
1804+55+1937+8823+4264

16883

In [25]:
1804+55+1937+8823+1477+3761

17857

In [26]:
df_output['zip_label'].unique()

array(['P', 'S', 'zip_in_10'], dtype=object)

In [28]:
df_output[df_output['zip_label']=="zip_in_10"]['SOTF'].unique()

array(['SOTF', 'Legacy'], dtype=object)